In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import glob
from sklearn.metrics import accuracy_score
from tqdm import tqdm

C:\Users\Gumpun\AppData\Local\Temp\ipykernel_8820\2203224730.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
!pip install attacut

In [3]:
!pip install pythainlp

Note: you may need to restart the kernel to use updated packages.


In [4]:
pos_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Positive.csv")
neg_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Negative.csv")
neu_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Neutrally.csv")

In [5]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
1447,ร***.,มาไม่ตรกปกไม่ชอบไม่ผ่านไม่อย่างสั่งแล้ว,Negative
227,Penpisut S.,ส่งสีเสื้อมาไม่ตรงกับที่สั่ง,Neutrally
799,0***0,วัสดุ ขนาด การออกแบบ ดีคะดีมากกกกกกกกกกกกกกกกก...,Positive
1217,Yui J.,ผ้านิ่มมาก แต่บาง เหมาะกับประเทศไทย,Positive
1461,samreong N.,คุณภาพผ้าพอใช้แต่สีผ้าดูจะสีซีไป,Neutrally
...,...,...,...
40,สุนันท์ ค.,สั่งสีครีมได้สีม่วงเฉ๊ย,Neutrally
1047,ฐิรนันทร์ ร.,ผ้าเบาใส่สบาย,Positive
1411,โภคิน,โฟมซักแห้งรองเท้าสีขาวใช้ได้ดีในระดับหนึ่งครับ...,Positive
399,*******038,เสื้อบางมากกก,Positive


In [6]:
category_counts = data['Sentiment'].value_counts()
category_counts

Sentiment
Positive     1499
Negative     1496
Neutrally    1496
Name: count, dtype: int64

In [7]:
!pip install pythainlp

In [8]:
from pythainlp.tokenize import word_tokenize

In [9]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)


,User,Message,Sentiment
1336,aiyarin,แย่มาก เลย คะ ส่ง กางเ ขาด มา ขาย ให้ ตัว ...,Negative
1024,พนมพร,ไม่ นุ่ม อย่าง ที่ คิด เลย จ้า พื้น แข็ง มา ...,Positive
1451,ช***า,สั่ง สี ดำ ได้ สี กรม กับ ส้ม อิฐ ส่วน สี ...,Neutrally
554,*******408,คุ้ม่า กับ รา คาที่ ดี เหมาะสำรับ สวมใ่ ใน ช...,Positive
842,0***2,เวลา เอา ผ้า ไป ปั่น เนื้อผา ยุ่ มาก หลุ...,Negative
1406,รังรักษ์ ส.,ส่ง ไว วส ผ้า บา ง ง ง ง ง ง ง คุณภาพ ตาม ...,Neutrally
1022,ชาติยา ล.,การเคลื่อนไหว ที่ เงียบส และ ราบื่น ใส่ ถ่าน...,Negative
282,จิรฐา ว.,รูปแบ สวยงาม ดู สะอาด ตา ดู ง่าย ดี ค่ะ ชอบ ...,Positive
90,Nu_nuy,สินค้า ไม่ โอเค ด้าย เย็บ หลุด อก,Neutrally
1154,จารุณี,ได้รับ สินค้า แล้ว ค่ะ สั่งซื้อ กางเ กีฬา ขา...,Neutrally


In [10]:
data["Message"]

0                                            รองเท้า นุ่ม
1                                              คุณภาพดี ๆ
2       จัดส่ง รวดเ็   ห่อ แพ ค สินค้า อย่าง ดี   บี เ...
3       รูปลักษณ์ภายนอ ดู ดีมาก   แต่ คุณภาพ แย่มาก ๆ ...
4       ส่ง เร็ว ตรง ตามกำหนด   วัน   สินค้า ครบ   ถูก...
                              ...                        
1495                                       กระท มี รอย บุ
1496    เนื้อผา ค่อนข้าง บาง   แต่ ลาย สวยงาม   ตรง ตา...
1497                       เ็ยบ เข็ม เดียว   ขาด ง่าย มาก
1498    มี ให้ เลือก สี หลากย   ผ้า เบา สบาย ใน การเคล...
1499    ผ้า สวย ทรง สวย แต่ เสียดา ไซส์ น้อย ไป หน่อย ...
Name: Message, Length: 4491, dtype: object

In [135]:
Message = data['Message'].values
Sentiment = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)
print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")

for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [137]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
train_sentences, test_sentences, train_labels, test_labels = train_test_split(Message, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [139]:
#โค้ดด้านบนน่าจะใช้สำหรับการหาความถี่ของคำที่เป็นเอกลักษณ์สำหรับแต่ละป้ายกำกับในชุดข้อมูลการจำแนกประเภทข้อความ

unique_labels = list(set(train_labels))
label_mapping = {label: i for i, label in enumerate(unique_labels)}
vec = CountVectorizer(max_features=5000, tokenizer=word_tokenize)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names_out()
X = X.toarray()
word_counts = {}
for label in unique_labels:
    word_counts[label] = defaultdict(lambda: 0)
for i in tqdm(range(X.shape[0])):
    label = label_mapping[train_labels[i]]
    words_in_sentence = word_tokenize(train_sentences[i])
    for word in words_in_sentence:
        if word in vocab:
            word_counts[label][word] += 1
for label, counts in word_counts.items():
    print(f"Label {label}: {len(counts)} unique words")

C:\Users\Gumpun\anaconda3\envs\py310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
100%|██████████| 4041/4041 [00:07<00:00, 539.70it/s]

Label 0: 1942 unique words
Label 1: 1743 unique words
Label 2: 1703 unique words


In [140]:
#ฟังก์ชันนี้ใช้สำหรับการทำนายความน่าจะเป็นของคำในเอกสารที่กำลังจะถูกจำแนก
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
    a = word_counts[text_label][word] + 1
    b = n_label_items[text_label] + len(vocab)
    return math.log(a/b)

#ฟังก์ชันนี้ใช้เพื่อแบ่งข้อมูลเป็นกลุ่มตามป้ายกำกับที่ให้มา
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]

    #print(data)
    return data

def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():

        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}

        words = set(text.strip().split(" ")) #split text with space
        #print(words)
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        #print(max(label_scores, key=label_scores.get))
        result.append(max(label_scores, key=label_scores.get))
    return result

In [141]:
labels = [0,1,2]
n_label_items, log_label_priors = fit(train_sentences, train_labels, labels)

In [142]:
# Predict
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
# Calculate accuracy
accuracy = accuracy_score(test_labels, pred)
print("Accuracy of prediction on test set : ", accuracy)

Accuracy of prediction on test set :  0.6444444444444445


In [145]:
from sklearn.metrics import classification_report
print(classification_report(test_labels,pred))

              precision    recall  f1-score   support

           0       0.65      0.66      0.66       150
           1       0.79      0.33      0.46       150
           2       0.60      0.95      0.74       150

    accuracy                           0.64       450
   macro avg       0.68      0.64      0.62       450
weighted avg       0.68      0.64      0.62       450



In [143]:
test_text = remove_duplicate_chars("ดีค่ะแต่บางไปหน่อย")
predicted_label = predict(n_label_items, vocab, word_counts, log_label_priors, labels, [test_text])

"""
0: Negative
1: Neutrally
2: Positive
"""

if predicted_label[0] == 0:
    print("The text is classified as negative.")
elif predicted_label[0] == 1:
    print("The text is classified as neutrally.")
else:
    print("The text is classified as positive.")

The text is classified as neutrally.


In [144]:
tokenized_text = test_text.split()
for word in tokenized_text:
    sentiment = predict(n_label_items, vocab, word_counts, log_label_priors, labels, [word])
    print(f"Text: {word}")
    print(f"Predicted sentiment: {sentiment[0]}")
    print("----------------------")

Text: ดี
Predicted sentiment: 2
----------------------
Text: ค่ะ
Predicted sentiment: 2
----------------------
Text: แต่
Predicted sentiment: 1
----------------------
Text: บาง
Predicted sentiment: 1
----------------------
Text: ไป
Predicted sentiment: 1
----------------------
Text: หน่อย
Predicted sentiment: 1
----------------------
